In [42]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'merge selected.ipynb',
 'Prepped Data',
 '개별공시지가정보',
 '개별공시지가정보 EDA.ipynb',
 '메인 데이터 EDA 1 - 주소 nan, top1 도로명.ipynb',
 '메인 데이터 베이스라인 준비 20211202.ipynb',
 '연속지적_서울',
 '연속지적도 SHP 파일 EDA.ipynb',
 '연속지적도로부터 좌표 구하기 - polylabel 활용.ipynb',
 '연속지적도출처 좌표자료 정리.ipynb',
 '위치정보관련',
 '좌표 데이터 prep 2 - up to date data.ipynb',
 '좌표 데이터 준비.ipynb',
 '토지특성정보 + 좌표 데이터 합침.ipynb',
 '토지특성정보 데이터 EDA, preprocessing, feature engineering - land_specs_last_baseline_20211116.csv.ipynb',
 '토지특성정보 데이터 EDA, preprocessing, feature engineering.ipynb',
 '토지특성정보 데이터 베이스라인 준비 20211116.ipynb']

In [3]:
os.listdir('./개별공시지가정보/')

['AL_11_D151_20211115']

In [4]:
basedir = './개별공시지가정보/'

In [5]:
foldernames = os.listdir(basedir)
foldernames

['AL_11_D151_20211115']

In [6]:
os.listdir(basedir + foldernames[0])

['AL_11_D151_20211115.csv']

In [8]:
official_price_df = pd.read_csv(basedir + foldernames[0] + '/AL_11_D151_20211115.csv', encoding='euc-kr')
print(official_price_df.shape)
official_price_df.head()

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(30302882, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15


In [9]:
seoul_df = official_price_df[official_price_df['법정동명'].str.contains('서울특별시')]
print(seoul_df.shape)
seoul_df.head()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [10]:
nan_df = official_price_df[official_price_df['법정동명'].isna()]
print(nan_df.shape)
nan_df.head()

(311, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
12653844,1132010601102640043,1132010601,NaN,1,일반,264-43,2000,1,990000,2000-06-30,Y,2021-11-15
12653845,1132010601106450042,1132010601,NaN,1,일반,645-42,2000,1,930000,2000-06-30,Y,2021-11-15
12653846,1132010601106460044,1132010601,NaN,1,일반,646-44,2000,1,1380000,2000-06-30,Y,2021-11-15
12653847,1132010601106470003,1132010601,NaN,1,일반,647-3,2000,1,1090000,2000-06-30,Y,2021-11-15
12653848,1132010601106470010,1132010601,NaN,1,일반,647-10,2000,1,920000,2000-06-30,Y,2021-11-15


In [11]:
pnu_df = pd.read_csv('./Prepped Data/서울특별시 건축물대장 법정동 코드정보.csv', encoding='euc-kr')
print(pnu_df.shape)
pnu_df.head()

(94131, 9)


,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일
0,27290,11800,617,대구광역시,달서구,유천동,유천동,20211129,20211129
1,41210,10600,655,경기도,광명시,일직동,일직동,20211129,99991231
2,27290,11900,617,대구광역시,달서구,대천동,유천동,20211129,20211129
3,27290,12000,617,대구광역시,달서구,월성동,유천동,20211129,20211129
4,27290,12100,617,대구광역시,달서구,월암동,유천동,20211129,20211129


In [12]:
official_price_df['법정동코드'].isna().sum()

0

In [17]:
def find_bubjungdong_name(x):
    # x is an int in 법정동코드 series
    
    x = str(x)
    
    시군구코드 = x[:5]
    법정동코드 = x[5:]
    
    target_df = pnu_df[(pnu_df['시군구코드'] == 시군구코드) & (pnu_df['법정동코드'] == 법정동코드)]
    
    if target_df.shape[0] == 0:
        print(x)
    
    시군구명 = target_df['시군구명'].iloc[0]
    법정동명 = target_df['법정동명'].iloc[0]
    
    return '서울특별시 ' + 시군구명 + ' ' + 법정동명

In [19]:
inv_df = pnu_df[pnu_df['시군구코드'] == '11320']
print(inv_df.shape)
inv_df.head()

(0, 9)


,시군구코드,법정동코드,행정동코드,시도명,시군구명,법정동명,행정동명,적용시작일,적용만료일


In [18]:
nan_df['법정동명'] = nan_df['법정동코드'].apply(find_bubjungdong_name)

1132010601


IndexError: single positional indexer is out-of-bounds

In [20]:
official_price_df = official_price_df.dropna(subset=['법정동명']).reset_index(drop=True)
print(official_price_df.shape)
official_price_df.head()

(30302571, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15


In [21]:
seoul_df = official_price_df[official_price_df['법정동명'].str.contains('서울특별시')]
print(seoul_df.shape)
seoul_df.head()

(30302571, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15


In [22]:
del seoul_df

In [23]:
recent_df = official_price_df[official_price_df['기준연도'] == 2021]
print(recent_df.shape)
recent_df.head()

(905792, 12)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자
31,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2021,1,5078000,2021-05-31,N,2021-11-15
44,1111010100100010001,1111010100,서울특별시 종로구 청운동,1,일반,1-1,2021,1,1569000,2021-05-31,N,2021-11-15
76,1111010100100010002,1111010100,서울특별시 종로구 청운동,1,일반,1-2,2021,1,2520000,2021-05-31,N,2021-11-15
108,1111010100100010003,1111010100,서울특별시 종로구 청운동,1,일반,1-3,2021,1,4872000,2021-05-31,N,2021-11-15
121,1111010100100010004,1111010100,서울특별시 종로구 청운동,1,일반,1-4,2021,1,1745000,2021-05-31,N,2021-11-15


In [53]:
del recent_df

In [24]:
official_price_df['기준월'].value_counts()

1    30198918
7       87805
9       11533
5        4315
Name: 기준월, dtype: int64

In [30]:
official_price_df['특수지구분명'].value_counts()

일반    29786365
산       516206
Name: 특수지구분명, dtype: int64

In [25]:
main_data_df = pd.read_csv('./Prepped Data/main_data_baseline_20211202.csv')
print(main_data_df.shape)
main_data_df.head()

(5188101, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [27]:
main_data_df['번지'].value_counts()

22         17353
17         17182
19         15878
316        14644
13         14245
           ...  
582-57         1
562-9          1
266-577        1
504-38         1
188-66         1
Name: 번지, Length: 60363, dtype: int64

In [29]:
main_data_df['번지'].str.contains('산').sum()

2409

In [31]:
def find_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return '글자없음'
    else:
        return x

In [32]:
main_data_df['특수지구분명'] = main_data_df['번지'].apply(find_letters)

In [33]:
main_data_df['특수지구분명'].value_counts()

글자없음    5180824
산          2409
BL         1553
가           560
Name: 특수지구분명, dtype: int64

In [43]:
official_price_df['글자'] = official_price_df['지번'].progress_apply(find_letters)

100%|██████████████████████████████████████████████████████████████████| 30302571/30302571 [01:29<00:00, 337828.33it/s]


In [44]:
official_price_df['글자'].value_counts()

글자없음    30302571
Name: 글자, dtype: int64

In [45]:
main_data_df['특수지구분명'] = main_data_df['특수지구분명'].apply(lambda x: '산' if x == '산' else '일반')

In [46]:
main_data_df['특수지구분명'].value_counts()

일반    5185692
산        2409
Name: 특수지구분명, dtype: int64

In [47]:
land_specs_df = pd.read_csv('./Prepped Data/land_specs_baseline_20211116.csv')
print(land_specs_df.shape)
land_specs_df.head()

(8549302, 16)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자
0,서울특별시 종로구 청운동,일반,1,1,2013,1,대,15622.1,제1종일반주거지역,지정되지않음,연립,평지,사다리형,소로한면,2840000,2017-10-17
1,서울특별시 종로구 청운동,일반,1,1,2014,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,2933000,2017-10-17
2,서울특별시 종로구 청운동,일반,1,1,2015,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3053000,2017-10-17
3,서울특별시 종로구 청운동,일반,1,1,2016,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3234000,2017-10-17
4,서울특별시 종로구 청운동,일반,1,1,2017,1,대,15622.1,제1종일반주거지역,지정되지않음,아파트,평지,사다리형,소로한면,3488000,2018-05-23


In [48]:
land_specs_df['대장구분명'].value_counts()

일반             8360481
산               152495
가지번              15241
블럭지번(지구)         11268
블럭지번              5322
기타지번              2949
블럭지번(지구_롯트)       1195
가지번(부번세분)          348
Name: 대장구분명, dtype: int64

In [49]:
land_specs_df['글자'] = land_specs_df['지번'].apply(find_letters)

In [50]:
land_specs_df['글자'].value_counts()

글자없음    8544963
단          1738
D           411
종           303
C           208
근생          153
상업          110
편익           99
연녹           94
행당           91
주            81
DS           74
준주           73
공공           58
B            58
Ds           50
주유           48
유치           40
군            37
단독           34
자족           32
아            32
사회           27
공익           26
공동           22
자동           19
고            19
종교           19
초            18
CP           16
어            16
S            16
청사           15
커뮤           14
T            14
공            12
주차           10
학            10
차고           10
체육           10
문            10
I            10
도로           10
의료            9
자원            9
근린            9
폐             9
경             8
화훼            6
학교            6
DP            6
업C            6
근             6
주유P           5
경관            5
상             5
주I            5
도서            4
단독R           4
전기            4
방수            4
가스Q           4
문화      

In [51]:
def remove_letters(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    y = x
    
    for i in range(10):
        x = x.replace(str(i), '')
    
    x = x.replace('-', '')
    
    if x == '':
        return y
    else:
        for v in x:
            y = y.replace(v, '')
        
        return y

In [52]:
main_data_df['번지'] = main_data_df['번지'].apply(remove_letters)

In [54]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,특수지구분명
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,일반
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,일반
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,일반
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반


In [55]:
official_price_df.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자,글자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15,글자없음
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15,글자없음
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15,글자없음
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15,글자없음
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15,글자없음


In [56]:
official_price_df.dtypes

고유번호       object
법정동코드       int64
법정동명       object
특수지구분코드     int64
특수지구분명     object
지번         object
기준연도        int64
기준월         int64
공시지가        int64
공시일자       object
표준지여부      object
데이터기준일자    object
글자         object
dtype: object

In [57]:
main_data_df['기준연도'] = main_data_df['계약년월'].apply(lambda x: int(str(x)[:4]))
main_data_df['기준월'] = main_data_df['계약년월'].apply(lambda x: int(str(x)[4:]))

In [60]:
def remove_first_and_last_spaces(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    if x[0] == ' ':
        x = x[1:]
    
    if x[-1] == ' ':
        x = x[:-1]
    
    return x

In [61]:
official_price_df['법정동명'] = official_price_df['법정동명'].apply(remove_first_and_last_spaces)

In [62]:
official_price_df.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자,글자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15,글자없음
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15,글자없음
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15,글자없음
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15,글자없음
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15,글자없음


In [63]:
main_data_df['시군구'] = main_data_df['시군구'].apply(remove_first_and_last_spaces)

In [64]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,특수지구분명,기준연도,기준월
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,일반,2011,1
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,일반,2011,1
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,일반,2011,2
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반,2011,2
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반,2011,2


In [65]:
main_data_df['주소'] = main_data_df['시군구'] + ' ' + main_data_df['특수지구분명'] + ' ' + main_data_df['번지']

In [66]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,특수지구분명,기준연도,기준월,주소
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,일반,2011,1,서울특별시 강남구 개포동 일반 655-2
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,일반,2011,1,서울특별시 강남구 개포동 일반 655-2
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,일반,2011,2,서울특별시 강남구 개포동 일반 655-2
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반,2011,2,서울특별시 강남구 개포동 일반 655-2
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,일반,2011,2,서울특별시 강남구 개포동 일반 655-2


In [67]:
official_price_df['주소'] = official_price_df['법정동명'] + ' ' + official_price_df['특수지구분명'] + ' ' + official_price_df['지번']

In [68]:
official_price_df.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자,글자,주소
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1


In [70]:
intersection_df = official_price_df[official_price_df['주소'].isin(main_data_df['주소'].unique())]
print(intersection_df.shape)
intersection_df.head()

(3432958, 14)


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공시지가,공시일자,표준지여부,데이터기준일자,글자,주소
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1990,1,750000,1990-08-30,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1991,1,980000,1991-06-29,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1992,1,980000,1992-06-01,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1993,1,980000,1993-05-22,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,1994,1,960000,1994-06-30,N,2021-11-15,글자없음,서울특별시 종로구 청운동 일반 1


In [71]:
%%time
intersection_df.to_csv('./Prepped Data/official_price_intersection_with_main_data_20211223.csv', index=False)

Wall time: 10.5 s


In [ ]:
%%time
coor_df = pd.read_csv('./Prepped Data/연속지적도출처_지번별좌표_본번부번정리_20211222.csv')
print(coor_df.shape)
coor_df.head()